# Overfitting Crab Age

*In which we practice one thing so much that we get worse at everything else.*


### Define Constants


In [ ]:
CACHE_FILE = '../cache/crabs.feather'
NEXT_NOTEBOOK = '../1-models/models.ipynb'

PREDICTION_TARGET = 'Age'    # 'Age' is predicted
DATASET_COLUMNS = ['Sex_F','Sex_M','Sex_I','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight',PREDICTION_TARGET]
REQUIRED_COLUMNS = [PREDICTION_TARGET]

MODEL_CHECKPOINT_FILE = '../cache/overfit_model.weights.h5'


### Import Libraries

PyTorch supports windows-native CUDA, but TensorFlow on CPU was faster for this task.


In [ ]:
from notebooks.time_for_crab.mlutils import display_df, plot_training_loss, score_combine, score_comparator, score_model

import keras

keras_backend = keras.backend.backend()
print(f'Keras version: {keras.__version__}')
print(f'Keras backend: {keras_backend}')
if keras_backend == 'tensorflow':
    import tensorflow as tf
    print(f'TensorFlow version: {tf.__version__}')
    print(f'TensorFlow devices: {tf.config.list_physical_devices()}')
elif keras_backend == 'torch':
    import torch
    print(f'Torch version: {torch.__version__}')
    print(f'Torch devices: {torch.cuda.get_device_name(torch.cuda.current_device())}')
    # torch supports windows-native cuda, but CPU was faster for this task

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.set_option('mode.copy_on_write', True)


### Load Data from Cache

In the [previous section](../0-eda/eda.ipynb), we saved the cleaned data to a cache file. Let's load it back.


In [ ]:
crabs = pd.read_feather(CACHE_FILE)
display_df(crabs)


## Overfitting Crab Age

![Large mud crab measure](https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/CSIRO_ScienceImage_10696_Mud_crabs_are_caught_measured_tagged_and_released_as_part_of_the_research_into_the_effectiveness_of_green_zones_in_Moreton_Bay.jpg/1920px-CSIRO_ScienceImage_10696_Mud_crabs_are_caught_measured_tagged_and_released_as_part_of_the_research_into_the_effectiveness_of_green_zones_in_Moreton_Bay.jpg)


### Overfitting Goals and Methods 

The goal here is to show how complex of a model it will take to overfit the data.

We will use the following methods to overfit the data:

1. **Linear Regression**: Gradually increase the complexity of a linear regression model.
2. **Model Stacking**: Stack models to overfit the model.

I am following the TensorFlow tutorial on [Linear Regression](https://www.tensorflow.org/tutorials/keras/regression#linear_regression_with_one_variable) to build the linear regression model.

> There are two steps in your single-variable linear regression model:  
>    1. Normalize the 'Horsepower' input features using the tf.keras.layers.Normalization preprocessing layer.
>    2. Apply a linear transformation ($y = mx + b$) to produce 1 output using a linear layer (tf.keras.layers.Dense).

Instead of 'Horsepower', we'll use 'Shell Weight' as our regression variable.


## Build the Shell Weight Model

This is a simple linear regression model that predicts the age of a crab based on its shell weight. It remains untrained.


#### Layer: Input Layer

This defines the shape of our input to the model.


In [ ]:
shell_weight_input = keras.layers.Input(shape=(1,))


#### Layer: Shell Weight Normalizer Layer

This is a quick and easy way to normalize our input data.

***Note**: In later steps, we will use a custom normalizer to show how it's done.*


In [ ]:
shell_weight = np.array(crabs['Shell Weight'])
shell_weight_normalizer = keras.layers.Normalization(axis=None)
shell_weight_normalizer.adapt(shell_weight)


#### Layer: Dense Layer

This is the layer that will perform the linear regression.


In [ ]:
shell_weight_dense = keras.layers.Dense(units=1)


### Architecture: Bringing it All Together

Now we'll put all the layers together to create the model. It is still going to be untrained.


In [ ]:
shell_weight_model = keras.Sequential([
    shell_weight_input,
    shell_weight_normalizer,
    shell_weight_dense
])

shell_weight_model.summary()


### Predictions: Pre-Training (For Science)

We don't expect good results here. This is just to get a baseline.



In [ ]:
shell_weight_target = np.array(crabs[PREDICTION_TARGET])
shell_weight_preds = shell_weight_model.predict(shell_weight).flatten()

print(shell_weight_target)
print(shell_weight_preds)


#### Scores: Naive Model

Throughout this notebook, we will use the following metrics to evaluate the regression model:

- **Mean Squared Error**
    - The best possible score is 0.0, lower values are better.
- **Mean Absolute Error**
    - The best possible score is 0.0, lower values are better. Less sensitive to outliers.
- **Explained Variance Score**
    - The best possible score is 1.0, lower values are worse.
- **R2 Score**
    - The best possible score is 1.0, lower values are worse.
- **Max Error**
    - The max error is the worst possible score. Domain-specific. 10 years is a lot for a crab.
 

In [ ]:
# Utility functions imported from mlutils.py
naive_scores_df = score_model(shell_weight_preds, shell_weight_target, index='naive')
naive_scores_df.head()


#### Observations: Naive Model

As expected, the untrained scores are terrible.


### Prepare the Shell Weight Model

Now it's finally time to get learning!

#### Compile

We will use the mean squared error as the loss function and the Adam optimizer.


In [ ]:
shell_weight_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_squared_error'
)


#### Checkpoint the Shell Weight Model

We want to save the model for later reference.


In [ ]:
shell_weight_checkpoint = keras.callbacks.ModelCheckpoint(
    MODEL_CHECKPOINT_FILE.replace('overfit', 'shell_weight'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)


### Train the Shell Weight Model

Let's train for 100 epochs.


In [ ]:
%%time
history = shell_weight_model.fit(
    x=crabs['Shell Weight'],
    y=crabs[PREDICTION_TARGET],
    epochs=100,
    verbose=0,
    validation_split=0.2,
    callbacks=[shell_weight_checkpoint]
)


#### Plot Training History


In [ ]:
history_df = pd.DataFrame(history.history)
history_df['epoch'] = history.epoch
history_df.tail()


#### Plot Loss History


In [ ]:
# Utility functions imported from mlutils.py
plot_training_loss(history)


#### View the Line of Best Fit

In [ ]:
x = np.linspace(crabs['Age'].min(), crabs['Age'].max(), 100)
y = shell_weight_model.predict(x).flatten()

plt.figure(figsize=(20, 10))
plt.scatter(crabs['Shell Weight'], crabs['Age'], label='Data')
plt.plot(x, y, color='red', label='Best Fit')
plt.xlabel('Shell Weight')
plt.ylabel('Age')
plt.legend()
plt.show()


### Predictions: Shell Weight Model

Now that we've trained the model, let's see how it performs.

We are hoping for almost 100% accuracy or zero error to achieve overfitting. However, we are not expecting this to happen just yet.

We have only trained the model on one feature, 'Shell Weight'. It is unlikely that just one feature (*except maybe the target itself*) will be able to overfit the data.


In [ ]:
shell_weight_target = np.array(crabs[PREDICTION_TARGET])
shell_weight_preds = shell_weight_model.predict(shell_weight).flatten()

print(shell_weight_target)
print(shell_weight_preds)


#### Scores: Shell Weight Model

Reminder of our metrics:

- **Mean Squared Error**
    - The best possible score is 0.0, lower values are better.
- **Mean Absolute Error**
    - The best possible score is 0.0, lower values are better. Less sensitive to outliers.
- **Explained Variance Score**
    - The best possible score is 1.0, lower values are worse.
- **R2 Score**
    - The best possible score is 1.0, lower values are worse.
- **Max Error**
    - The max error is the worst possible score. Domain-specific. 10 years is a lot for a crab.
 

In [ ]:
# Utility functions imported from mlutils.py
shell_weight_scores_df = score_model(shell_weight_preds, shell_weight_target, index='shell_weight')

leaderboard_df = score_combine(naive_scores_df, shell_weight_scores_df)

leaderboard_df.head()


### Comparison: Naive vs Shell Weight

In [ ]:
leaderboard_df.plot(kind='bar', title='Naive vs Trained Model Scores', figsize=(20, 10))


#### Score-by-Score Comparison: Naive vs Shell Weight

Break down the bar charts into one for each metric.


In [ ]:
# Utility functions imported from mlutils.py
score_comparator(naive_scores_df, shell_weight_scores_df, train_label='Naive', test_label='Trained')


### Observations: Shell Weight Model

The scores are much better after training! But not even close to overfitting the data.

Shell Weight alone must not be a good predictor of crab age.


## Build the Feature-Rich Model

Moving on to [linear regression with multiple variables](https://www.tensorflow.org/tutorials/keras/regression#linear_regression_with_multiple_inputs).

In order to overfit, we're going to need to give the model more features from the dataset.

Shell Weight alone is not going to cut it. We need to increase the complexity to a point where the model is just memorizing the training data.

We'll increase the input dimensions to the number of crab features. We exclude the target column from this count.



In [ ]:
# layer: input
overfit_input = keras.layers.Input(shape=(len(crabs.columns) - 1,))

# layer: normalizer
overfit_normalizer = keras.layers.Normalization(axis=-1)
overfit_normalizer.adapt(np.array(crabs.drop(columns=[PREDICTION_TARGET])))

# layer: dense (linear regression)
overfit_dense = keras.layers.Dense(units=1)

# architecture:
#   input -> normalizer -> dense
overfit_model = keras.Sequential([
    overfit_input,
    overfit_normalizer,
    overfit_dense
])

overfit_model.summary()


### Prepare the Feature-Rich Model

#### Compile

We will use the mean squared error as the loss function and the Adam optimizer.


In [ ]:
overfit_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_squared_error'
)


#### Checkpoint the Feature-Rich Model

We want to save the model for later reference.


In [ ]:
overfit_checkpoint = keras.callbacks.ModelCheckpoint(
    MODEL_CHECKPOINT_FILE,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)


### Train the Feature-Rich Model

Let's train for 100 epochs.

In [ ]:
%%time
history = overfit_model.fit(
    x=crabs.drop(columns=[PREDICTION_TARGET]),
    y=crabs[PREDICTION_TARGET],
    epochs=100,
    verbose=0,
    validation_split=0.2,
    callbacks=[overfit_checkpoint]
)


#### Plot the Loss History

In [ ]:
# Utility functions imported from mlutils.py
plot_training_loss(history)


### Predictions: Feature-Rich Model

We are hoping it did *too* well.


In [ ]:
overfit_target = np.array(crabs[PREDICTION_TARGET])
overfit_preds = overfit_model.predict(crabs.drop(columns=[PREDICTION_TARGET])).flatten()

print(overfit_target)
print(overfit_preds)


#### Scores: Feature-Rich Model

Metrics used:

- **Mean Squared Error**
    - The best possible score is 0.0, lower values are better.
- **Mean Absolute Error**
    - The best possible score is 0.0, lower values are better. Less sensitive to outliers.
- **Explained Variance Score**
    - The best possible score is 1.0, lower values are worse.
- **R2 Score**
    - The best possible score is 1.0, lower values are worse.
- **Max Error**
    - The max error is the worst possible score. Domain-specific. 10 years is a lot for a crab.


In [ ]:
# Utility functions imported from mlutils.py
overfit_scores_df = score_model(overfit_preds, overfit_target, index='overfit')

leaderboard_df = score_combine(leaderboard_df, overfit_scores_df)

leaderboard_df.head()


### Comparison: Shell Weight vs Feature-Rich

We'll see if this feature-rich model is any better than the Shell Weight model.


In [ ]:
leaderboard_df.drop('naive').plot(kind='bar', title='Shell Weight vs All Feature Model Scores', figsize=(20, 10))


#### Score-by-Score Comparison: Shell Weight vs Feature-Rich

Break down the bar charts into one for each metric.


In [ ]:
# Utility functions imported from mlutils.py
score_comparator(shell_weight_scores_df, overfit_scores_df, train_label='Shell Weight', test_label='Feature-Rich')


### Observations: Feature-Rich Model

The scores are better than the Shell Weight model, but not by much. We must go deeper!


## Don't Save this Data

We don't want our over-trained model to leak into the [next step](../1-models/models.ipynb).


## Onwards to Model Selection

See the [next section](../1-models/models.ipynb) for model selection.


In [ ]:
get_ipython().run_line_magic('notebook', NEXT_NOTEBOOK)
